In [ ]:
#| default_exp tune
#| default_cls_lvl 3

In [ ]:
#| export
from tsfast.data import *
from tsfast.models import *
from tsfast.learner import *

from fastai.basics import *
from fastai.callback.core import Callback
# from fastai.callback.schedule import *
# from fastai.callback.rnn import *
# from fastai.callback.tracker import *

import ray
from ray import tune
from ray.tune.schedulers import *
from ray.tune.experiment.trial import ExportFormat
from ray.tune import Checkpoint

In [ ]:
from nbdev.config import get_config

In [ ]:
project_root = get_config().config_file.parent
f_path = project_root / 'test_data/WienerHammerstein'

In [ ]:
hdf_files = L([f for f in get_hdf_files(f_path) if '_test.hdf5' not in str(f)])
tfm_src = CreateDict([DfHDFCreateWindows(win_sz=400,stp_sz=100,clm='u')])
dls = DataBlock(blocks=(SequenceBlock.from_hdf(['u'],TensorSequencesInput),
                        SequenceBlock.from_hdf(['y'],TensorSequencesOutput)),
                get_items=tfm_src,
                splitter=ApplyToDict(FuncSplitter(lambda o: 'valid' in str(o)))).dataloaders(hdf_files)

## optimizer core

First we need a log uniform distibution for variables with vast value ranges

In [ ]:
#| export
def log_uniform(min_bound, max_bound, base=10):
    '''uniform sampling in an exponential range'''
    logmin = np.log(min_bound) / np.log(base)
    logmax = np.log(max_bound) / np.log(base)
    def _sample():
        return base**(np.random.uniform(logmin, logmax))
    return _sample

In [ ]:
[log_uniform(1e-8, 1e-2)() for _ in range(5)]

[4.591524992137234e-08,
 0.003755410605938488,
 2.920688605923387e-07,
 3.4750213799838236e-06,
 2.1312097874133118e-08]

In [ ]:
#| export
class LearnerTrainable(tune.Trainable):

    def setup(self, config):
        self.create_lrn = ray.get(config['create_lrn'])
        self.dls = ray.get(config['dls'])

        self.lrn = self.create_lrn(self.dls,config)

    def step(self):
        with self.lrn.no_bar(): self.lrn.fit(1)
        train_loss,valid_loss,rmse = self.lrn.recorder.values[-1]
        result = {'train_loss': train_loss,
                'valid_loss': valid_loss,
                'mean_loss': rmse}
        return result

    def save_checkpoint(self, tmp_checkpoint_dir):
        checkpoint_path = os.path.join(tmp_checkpoint_dir, "model.pth")
        torch.save(self.lrn.model.state_dict(), checkpoint_path)
        return tmp_checkpoint_dir

    def load_checkpoint(self, tmp_checkpoint_dir):
        checkpoint_path = os.path.join(tmp_checkpoint_dir, "model.pth")
        self.lrn.model.load_state_dict(torch.load(checkpoint_path))

    def _export_model(self, export_formats, export_dir):
        if export_formats == [ExportFormat.MODEL]:
            path = os.path.join(export_dir, "exported_model")
            torch.save(self.lrn.model.state_dict(), path)
            return {ExportFormat.MODEL: path}
        else:
            raise ValueError("unexpected formats: " + str(export_formats))

    # the learner class will be recreated with every perturbation, saving the model
    # that way the new hyperparameter will be applied
    def reset_config(self, new_config):
        self.lrn = self.create_lrn(self.dls,new_config)
        self.config = new_config
        return True

In [ ]:
#| export
from multiprocessing.managers import SharedMemoryManager
def stop_shared_memory_managers(obj):
    """
    Iteratively finds and stops all SharedMemoryManager instances contained within the provided object.
    """
    visited = set()  # Track visited objects to avoid infinite loops
    stack = [obj]  # Use a stack to manage objects to inspect

    while stack:
        current_obj = stack.pop()
        obj_id = id(current_obj)

        if obj_id in visited:
            continue  # Skip already visited objects
        visited.add(obj_id)

        # Check if the current object is a SharedMemoryManager and stop it
        if isinstance(current_obj, SharedMemoryManager):
            current_obj.shutdown()
            continue

        # If it's a collection, add its items to the stack. Otherwise, add its attributes.
        if isinstance(current_obj, dict):
            stack.extend(current_obj.keys())
            stack.extend(current_obj.values())
        elif isinstance(current_obj, (list, set, tuple)):
            stack.extend(current_obj)
        elif hasattr(current_obj, '__dict__'):  # Check for custom objects with attributes
            stack.extend(vars(current_obj).values())

In [ ]:
#| export
import gc
def learner_optimize(config):
    try:
        create_lrn = ray.get(config['create_lrn'])
        dls = ray.get(config['dls'])
        
        #Scheduling Parameters for training the Model
        lrn_kwargs = {'n_epoch':100,'pct_start':0.5}
        for attr in ['n_epoch','pct_start']:
            if attr in config: lrn_kwargs[attr] = config[attr]
    
        lrn = create_lrn(dls,config)
        
        # load checkpoint data if provided
        checkpoint: tune.Checkpoint = tune.get_checkpoint()
        if checkpoint:
            with checkpoint.as_directory() as checkpoint_dir:
                lrn.model.load_state_dict(torch.load(checkpoint_dir + 'model.pth'))
        
        lrn.lr = config['lr'] if 'lr' in config else 3e-3
        lrn.add_cb(CBRayReporter() if 'reporter' not in config else ray.get(config['reporter'])())
        with lrn.no_bar(): 
            ray.get(config['fit_method'])(lrn,**lrn_kwargs)
    finally:
        #cleanup shared memory even when earlystopping occurs
        if 'lrn' in locals():
            stop_shared_memory_managers(lrn)
            del lrn
            gc.collect()

In [ ]:

class TrainSpecificEpoch(Callback):
    "Skip training up to `epoch`"
    order = 70
    
    def __init__(self, epoch:int):
        self._skip_to = epoch

    def before_epoch(self):
        print(self.epoch)
        # if self.epoch < self._skip_to:
        #     raise CancelEpochException
        # if self.epoch > self._skip_to:
        # raise CancelFitException

In [ ]:
class TrainableModel(tune.Trainable):
    def setup(self, config):
        # Assuming create_lrn and dls are accessible here or passed in config
        self.create_lrn = ray.get(config['create_lrn'])
        self.dls = ray.get(config['dls'])
        self.config = config

        self.lrn = self.create_lrn(self.dls, config)

        self.lrn.lr = config['lr'] if 'lr' in config else 3e-3
        if 'wd' in config: self.lrn.wd = config['wd']
        self._setup_callbacks()

        if 'reporter' not in self.config:
            self.lrn.add_cb(CBRayReporter())
        else:
            self.lrn.add_cb(ray.get(self.config['reporter'])())

        if self.lrn.opt is None: self.lrn.create_opt()
        self.lrn.opt.set_hyper('lr', self.lrn.lr)
        lr = np.array([h['lr'] for h in self.lrn.opt.hypers])
        pct_start = config['pct_start'] if 'pct_start' in config else 0.3
        self.n_epoch = config['n_epoch'] if 'n_epoch' in config else 10
        lr_scheds = {'lr': combined_cos(pct_start, lr, lr, lr/div_final)}
        self.steps=0

    def step(self):

        self.fit(self.n_epoch, cbs=TrainSpecificEpoch(self.steps)+ParamScheduler(scheds)+L(cbs), wd=wd)
        self.steps += 1

        
        scores = self.lrn.recorder.values[-1]
        metrics = {
            'train_loss': scores[0],
            'valid_loss': scores[1]
        }        
        for metric,value in zip(self.learn.metrics,scores[2:]):
            m_name = metric.name if hasattr(metric,'name') else str(metric)
            metrics[m_name] = value
        return metrics

    def save_checkpoint(self, checkpoint_dir):
        file = os.path.join(temp_checkpoint_dir,'model.pth')
        save_model(file, self.learn.model,opt=None) 

    def load_checkpoint(self, checkpoint_path):
        self.lrn.model.load_state_dict(torch.load(checkpoint_dir + 'model.pth'))


In [ ]:
class TrainableModel(tune.Trainable):
    def setup(self, config):
        # Assuming create_lrn and dls are accessible here or passed in config
        self.create_lrn = ray.get(config['create_lrn'])
        self.dls = ray.get(config['dls'])
        self.config = config
        self.lrn_kwargs = {'n_epoch': 100, 'pct_start': 0.5}

        for attr in ['n_epoch', 'pct_start']:
            if attr in config:
                self.lrn_kwargs[attr] = config[attr]

        self.lrn = self.create_lrn(self.dls, config)
        self.lrn.lr = config['lr'] if 'lr' in config else 3e-3


    def step(self):
        print(self.iteration)
        # fit_kwargs = {**self.lrn_kwargs,**{'cbs':TrainSpecificEpoch(self.iteration)}}
        # fit_kwargs = {**self.lrn_kwargs,**{'cbs':SkipToEpoch(self.iteration)}}
        # fit_kwargs = self.lrn_kwargs
        with self.lrn.no_bar(): 
            # ray.get(self.config['fit_method'])(self.lrn,**fit_kwargs)
            # self.lrn.fit_flat_cos(**fit_kwargs)
            self.lrn.fit_flat_cos(self.lrn_kwargs['n_epoch'],cbs=TrainSpecificEpoch(self.iteration))

        
        metrics = {
            'train_loss': 1,#scores[0],
            'valid_loss': 1,#scores[1],
             tune.result.DONE: self.iteration >= self.lrn_kwargs['n_epoch']-1
        }  
        
        # scores = self.lrn.recorder.values[-1]
        # metrics = {
        #     'train_loss': scores[0],
        #     'valid_loss': scores[1],
        #      tune.result.DONE: self.epoch_iter >= self.lrn_kwargs['n_epoch']
        # }        
        # for metric,value in zip(self.lrn.metrics,scores[2:]):
        #     m_name = metric.name if hasattr(metric,'name') else str(metric)
        #     metrics[m_name] = value
        return metrics

    def save_checkpoint(self, checkpoint_dir):
        file = os.path.join(temp_checkpoint_dir,'model.pth')
        save_model(file, self.learn.model,opt=None) 

    def load_checkpoint(self, checkpoint_path):
        self.lrn.model.load_state_dict(torch.load(checkpoint_dir + 'model.pth'))


In [ ]:
dls = DataBlock(blocks=(SequenceBlock.from_hdf(['u'],TensorSequencesInput),
                    SequenceBlock.from_hdf(['y'],TensorSequencesOutput)),
            get_items=tfm_src,
            splitter=ApplyToDict(FuncSplitter(lambda o: 'valid' in str(o)))).dataloaders(hdf_files)

The mutation config dictionary consists of functions that sample from a distribution. In order to retrieve a dictionary with one realisation we need the function sample_config

In [ ]:
#| export
def sample_config(config):
    ret_conf = config.copy()
    for k in ret_conf:
        ret_conf[k]=ret_conf[k]()
    return ret_conf

In [ ]:
#| export
class CBRayReporter(Callback):
    "`Callback` reports progress after every epoch to the ray tune logger"
    
    order=70 #order has to be >50, to be executed after the recorder callback

    def after_epoch(self):
        # train_loss,valid_loss,rmse = self.learn.recorder.values[-1]
        # metrics = {
        #     'train_loss': train_loss,
        #     'valid_loss': valid_loss,
        #     'mean_loss': rmse,
        # }
        scores = self.learn.recorder.values[-1]
        metrics = {
            'train_loss': scores[0],
            'valid_loss': scores[1]
        }
        for metric,value in zip(self.learn.metrics,scores[2:]):
            m_name = metric.name if hasattr(metric,'name') else str(metric)
            metrics[m_name] = value

        with tempfile.TemporaryDirectory() as temp_checkpoint_dir:
            file = os.path.join(temp_checkpoint_dir,'model.pth')
            #the model has to be saved to the checkpoint directory on creation
            #that is why a seperate callback for model saving is not trivial
            save_model(file, self.learn.model,opt=None) 
            ray.tune.report(metrics, checkpoint=Checkpoint.from_directory(temp_checkpoint_dir))

In [ ]:
#| export
class HPOptimizer():
    def __init__(self,create_lrn,dls):
        self.create_lrn = create_lrn
        self.dls = dls
        self.analysis = None
    
    @delegates(ray.init)
    def start_ray(self,**kwargs):
        ray.shutdown()
        ray.init(**kwargs)
        
    def stop_ray(self):
        ray.shutdown()
        
    
        

    @delegates(tune.run, keep=True)
    def optimize(self,config,optimize_func=learner_optimize,resources_per_trial={"gpu": 1.0},verbose=1,**kwargs):
        config['create_lrn'] = ray.put(self.create_lrn)
        #dls are large objects, letting ray handle the copying process makes it much faster
        config['dls'] = ray.put(self.dls) 
        if 'fit_method' not in config: config['fit_method'] = ray.put(Learner.fit_flat_cos)

        self.analysis = tune.run(
            optimize_func,
            config=config,
            resources_per_trial=resources_per_trial,
            verbose=verbose,
            **kwargs)
        return self.analysis
        
    @delegates(tune.run, keep=True)
    def optimize_pbt(self,opt_name,num_samples,config,mut_conf,perturbation_interval=2,
                 stop={"training_iteration": 40 },
                 resources_per_trial={"gpu": 1 },
                 resample_probability=0.25,
                 quantile_fraction=0.25,
                 **kwargs):
        self.mut_conf = mut_conf
        
        config['create_lrn'] = ray.put(self.create_lrn)
        #dls are large objects, letting ray handle the copying process makes it much faster
        config['dls'] = ray.put(self.dls) 
        

        
        scheduler = PopulationBasedTraining(
        time_attr="training_iteration",
        metric="mean_loss",
        mode="min",
        perturbation_interval=perturbation_interval,
        resample_probability=resample_probability,
        quantile_fraction=quantile_fraction,
        hyperparam_mutations=mut_conf)
        
        self.analysis = tune.run(
            LearnerTrainable,
            name=opt_name,
            scheduler=scheduler,
            reuse_actors=True,
            verbose=1,
            stop=stop,
            checkpoint_score_attr="mean_loss",
            num_samples=num_samples,
            resources_per_trial=resources_per_trial,
            config=config,
            **kwargs)
        return self.analysis
    
    def best_model(self):
        if self.analysis is None: raise Exception
        model = self.create_lrn(self.dls,sample_config(self.mut_conf)).model
        f_path = ray.get(self.analysis.get_best_trial('mean_loss',mode='min').checkpoint.value)
        model.load_state_dict(torch.load(f_path))
        return model

### Test Population Based Training

In [ ]:
def create_lrn(dls,config):
    lr = config['lr']
    alpha = config['alpha']
    beta = config['beta']
    weight_p = config['weight_p']
    
    lrn = RNNLearner(dls)
    lrn.lr = lr
    return lrn

In [ ]:
config={
            "lr": tune.loguniform(1e-2, 1e-4),
            "alpha": tune.loguniform(1e-5, 10),
            "beta": tune.loguniform(1e-5, 10),
            "weight_p": tune.uniform(0, 0.5)}
mut_conf = {# distribution for resampling
            "lr": log_uniform(1e-8, 1e-2),
            "alpha": log_uniform(1e-5, 10),
            "beta": log_uniform(1e-5, 10),
            "weight_p": lambda: np.random.uniform(0, 0.5)}

hp_opt = HPOptimizer(create_lrn,dls)
# hp_opt.start_ray()
# hp_opt.optimize_pbt('pbt_test',4,config,mut_conf,perturbation_interval=1,
#                  stop={"training_iteration": 1 },
#                  resources_per_trial={"gpu": 0.5},
#                  storage_path=str(Path.home() / 'ray_results'))#no cpu count is necessary

In [ ]:
#hp_opt.best_model()

### Test Grid Search

In [ ]:
# dls.cpu()

In [ ]:
def create_lrn(dls,config):
    dls = DataBlock(blocks=(SequenceBlock.from_hdf(['u'],TensorSequencesInput),
                        SequenceBlock.from_hdf(['y'],TensorSequencesOutput)),
                get_items=tfm_src,
                splitter=ApplyToDict(FuncSplitter(lambda o: 'valid' in str(o)))).dataloaders(hdf_files)
    lrn = RNNLearner(dls,hidden_size=config['hidden_size'],metrics=[fun_rmse,mse])
    return lrn

In [ ]:
hp_opt = HPOptimizer(create_lrn,None)

In [ ]:
search_space = {
    "hidden_size": tune.grid_search([10,20,50,100]),
    'n_epoch':10
}

In [ ]:
# hp_opt.optimize(optimize_func=TrainableModel,
#                 resources_per_trial={"cpu": 4},
#                 config=search_space)

In [ ]:
# hp_opt.analysis.get_best_config('mean_loss',mode='min')

### Test Random Search

In [ ]:
#| include: false
import nbdev; nbdev.nbdev_export()